test sentiment analysis

In [36]:
# all imports
import json
import tensorflow as tf
import nltk
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import math
import re
import os
import numpy as np
import json

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [37]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [38]:
# path to sarcasm dataset path in json format
dataset_path = '/gdrive/MyDrive/DL_mini_project/datasets/sarcasm.json'

if os.path.exists(dataset_path):
  print('the path exists')
else:
  print('Ensure the path is correct.')

the path exists


In [40]:
# load the test set
test_set_path = '/gdrive/MyDrive/DL_mini_project/datasets/test_set_processed.npz'
test_data = np.load(test_set_path)

In [41]:
# sepearte it into features and labels
test_padded, test_labels = test_data['test_padded'], test_data['test_labels']

In [42]:
# print shape
print(test_padded.shape, test_labels.shape)

(4007, 200) (4007,)


In [48]:
# load teh model from teh json file
model_path = '/gdrive/MyDrive/DL_mini_project/Task3/A/model_mlp_architecture.json'


with open(model_path, 'r') as json_file:
    loaded_model_json = json_file.read()

# Create a new model using the loaded architecture
model_mlp = tf.keras.models.model_from_json(loaded_model_json)

# Compile the loaded model (adjust compilation configuration as needed)
model_mlp.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [44]:
model_mlp.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 200, 100)          500000    
                                                                 
 global_average_pooling1d_2  (None, 100)               0         
  (GlobalAveragePooling1D)                                       
                                                                 
 dense_12 (Dense)            (None, 128)               12928     
                                                                 
 dropout_6 (Dropout)         (None, 128)               0         
                                                                 
 dense_13 (Dense)            (None, 64)                8256      
                                                                 
 dense_14 (Dense)            (None, 32)                2080      
                                                      

In [49]:
weights_path = "/gdrive/MyDrive/DL_mini_project/Task3/A/weights/model_mlp_weights.h5"
embeddings_matrix_path = "/gdrive/MyDrive/DL_mini_project/Task3/A/weights/vecs.tsv"

In [51]:
model_mlp.load_weights(weights_path)

# Load the pre-trained embeddings from the TSV file
embeddings_matrix = np.loadtxt(embeddings_matrix_path, delimiter='\t')

embedding_layer = model_mlp.get_layer('embedding')
embedding_layer.set_weights([embeddings_matrix])

In [52]:
# predict on test_set

# Predict on the test set
predictions = model_mlp.predict(test_padded)

# Convert predictions to binary (0 or 1) based on a threshold (e.g., 0.5)
binary_predictions = (predictions > 0.55).astype(int)

# Calculate accuracy
accuracy = np.mean(binary_predictions == test_labels)

print(f"Accuracy on the test set: {accuracy * 100:.2f}%")

126/126 [==============================] - 1s 3ms/step
Accuracy on the test set: 51.49%
